In [1]:
import time
import pymongo
import os
import numpy as np
import pandas as pd
import requests
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import magic

# How many documents do we have to scrape and download?

In [27]:
""" A util for making a connection to mongo """
def _connect_mongo(host, port, username, password, db):
    try:
        if username and password:
            mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
            conn = pymongo.MongoClient(mongo_uri)
            
        else:
            conn = pymongo.MongoClient(host, port)

        print "Connected successfully!!!"
        return conn[db]
    
    except pymongo.errors.ConnectionFailure, e:
        print "Could not connect to MongoDB: %s" % e 

def insert_folio(db,collection,insert_query={},
                 host='localhost',port=27017,username=None, password=None,):
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    return db
    
""" send a query and reutnr the count """   
def mongo_count(db,collection, query={}, host='localhost', port=27017, 
               username=None, password=None, no_id=True):
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # With query get the count
    count = db[collection].find(query).count()
    return count

""" Read from Mongo and Store into Pandas DataFrame """
def read_mongo(db, collection, query={}, nb_limit=None, host='localhost', port=27017,
               username=None, password=None, no_id=True):

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    if nb_limit == None:
        # just query
        cursor = db[collection].find(query)
    else:
        cursor = db[collection].find(query).limit(nb_limit)
    
    df = pd.DataFrame(list(cursor))
    
    if no_id :
        del df['_id']
    return df


def _download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True, verify = False)
    with open('./'+local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename


# we need to change the infilename into something else for the respuestas
def download_file(url,filename,path,extension=""):
    infilename = _download_file(url)
    newname = os.path.splitext(path+filename)[0]+extension
    # yes
    output = os.rename(infilename, newname)
    print "File is located at:",newname
    return str(newname)
    
    
"""Downloads a stack of urls"""
def download_stack_urls(stackOfUrls,fileNameBase,path):
    count = 0
    length = len(stackOfUrls)
    for url in stackOfUrls:
        r = requests.get(url, verify = False)
        if r.status_code == 200:
            download_file_raw(url,str(fileNameBase)+str(count),path)
        else:
            print r.status_code,"for html url"
            print url
        count+=1
        print "Downloaded",count,"out of",length
        time.sleep(1)
        
def request_to_db(request,db):
    pass

def update_request(request,db):
    pass

def uploadRequest(db):
    pass

"""Determine the file_extension of a magicFile"""
def what_is_file_extension(magicFile):
    import re
    r = magic.from_file(magicFile)
    if re.search("Microsoft Word",r):
        return ".docx"
    if re.search("Composite Document File V2 Document",r):
        return ".doc"
    if re.search("PDF document",r):
        return ".pdf"
    if re.search("Rich Text Format data",r):
        return ".rtf"
    if re.search("HTML",r):
        return ".html"
    if re.search("Zip",r):
        return ".zip"
    if re.search("ASCII text",r):
        return ".txt"
    else:
        return None

In [3]:
df = read_mongo('polisci_test05Nov16',
                'folios',
                {'ARCHIVORESPUESTA' : {'$regex' : 'respuesta=' }},
                no_id=False
               )

Connected successfully!!!


In [4]:
df.head()

,ARCHIVOADJUNTOSOLICITUD,ARCHIVORESPUESTA,DEPENDENCIA,DESCRIPCIONSOLICITUD,ESTATUS,FECHARESPUESTA,FECHASOLICITUD,FOLIO,LOCALIDAD,MEDIOENTRADA,OTROSDATOS,PDFOCR,RESPUESTA,SECTOR,TIPOSOLICITUD,_id,path_to_file,year
0,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE GOBERNACIÓN (INCLUYE LA ENTONCES...,cuenta publica municipal 2013-2014 del municip...,Electrónica,2015-01-08,2014-12-20 21:25:00,'0000400000115,"{u'CODIGOPOSTAL': u'75412', u'PAIS': u'México'...",Electrónica,NaN,0,La solicitud no corresponde al marco de la Ley,Gobernación,Información Pública,581e7b270fa8cb6876cc06c9,None,2014
1,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE GOBERNACIÓN (INCLUYE LA ENTONCES...,Convenios (incluyendo sus anexos) celebrados c...,Electrónica,2015-01-09,2014-12-22 09:03:00,'0000400000215,"{u'CODIGOPOSTAL': u'16600', u'PAIS': u'México'...",Electrónica,NaN,0,La información está disponible públicamente,Gobernación,Información Pública,581e7b270fa8cb6876cc06ca,None,2014
2,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE GOBERNACIÓN (INCLUYE LA ENTONCES...,Preguntas para una solicitud de información a ...,Electrónica,2015-01-13,2014-12-22 11:56:00,'0000400000315,"{u'CODIGOPOSTAL': u'9227', u'PAIS': u'México',...",Electrónica,NaN,0,Requerimiento de información adicional,Gobernación,Información Pública,581e7b270fa8cb6876cc06cb,None,2014
3,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE GOBERNACIÓN (INCLUYE LA ENTONCES...,Documentos de los contratos realizados con la ...,Electrónica,2015-03-05,2014-12-22 13:49:00,'0000400000615,"{u'CODIGOPOSTAL': u'9780', u'PAIS': u'México',...",Electrónica,NaN,0,Entrega de información en medio electrónico,Gobernación,Información Pública,581e7b270fa8cb6876cc06ce,None,2014
4,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE GOBERNACIÓN (INCLUYE LA ENTONCES...,Solicitó saber cual era el salario neto de la ...,Electrónica,2015-02-03,2014-12-22 14:27:00,'0000400000715,"{u'CODIGOPOSTAL': u'25000', u'PAIS': u'México'...",Electrónica,En la secretaria de Gobernación o alguna de su...,0,Entrega de información en medio electrónico,Gobernación,Información Pública,581e7b270fa8cb6876cc06cf,None,2014


In [5]:
# the path to the directory 
attachment_directory = '../code/'

In [6]:
test_document = df.iloc[0]

In [7]:
object_id = test_document['_id']
year = test_document['year']
url = test_document['ARCHIVORESPUESTA']
print url

https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeAdjuntoSeguimientoGenerator.action?folio=0000400000115&respuesta=3


In [8]:
object_id

ObjectId('581e7b270fa8cb6876cc06c9')

In [9]:
path_year = str(attachment_directory)+str(year)+'/'
os.stat(path_year)

posix.stat_result(st_mode=16877, st_ino=74347143, st_dev=16777220, st_nlink=54, st_uid=501, st_gid=20, st_size=1836, st_atime=1478487788, st_mtime=1478487788, st_ctime=1478487788)

In [10]:
path = str(attachment_directory)+str(year)+'/'+str(object_id)
os.stat(path)

OSError: [Errno 2] No such file or directory: '../code/2014/581e7b270fa8cb6876cc06c9'

OH NO! No path. lets make it:

In [11]:
os.mkdir(str(attachment_directory)+str(year)+'/'+str(object_id))
os.stat(path)

posix.stat_result(st_mode=16877, st_ino=74696920, st_dev=16777220, st_nlink=2, st_uid=501, st_gid=20, st_size=68, st_atime=1478487874, st_mtime=1478487874, st_ctime=1478487874)

In [12]:
download_file(url,'/afile',path)

File is located at: ../code/2014/581e7b270fa8cb6876cc06c9/afile


In [13]:
# what is the extension of file
what_is_file_extension('../code/2014/581e7b270fa8cb6876cc06c9/afile')

'.html'

In [16]:
renamed_file = os.path.join(path,'afile'+'.html')
os.rename('../code/2014/581e7b270fa8cb6876cc06c9/afile',renamed_file)

In [56]:
def a_function(document,attachment_directory,afilename):
    object_id = document['_id']
    year = document['year']
    url = document['ARCHIVORESPUESTA']
    path_year = str(attachment_directory)+str(year)+'/'
    path_to_download = str(attachment_directory)+str(year)+'/'+str(object_id) 
    
    try:
        os.stat(path_to_download)
    except:
        os.mkdir(path_to_download)
    
    downloaded_file = download_file(url,afilename,path_to_download)
    extension = what_is_file_extension(str(downloaded_file))
    renamed_file = os.path.join(path_to_download,afilename+str(extension))
    print renamed_file
    print path_to_download
    try:
        os.rename(path_to_download+afilename,renamed_file)
        print "successfuly renamed"
    except:
        print "unsuccessful rename"
        return renamed_file
    
    return renamed_file

In [57]:
a_function(df.iloc[1],attachment_directory,'attachment')

File is located at: ../code/2014/581e7b270fa8cb6876cc06caattachment
../code/2014/581e7b270fa8cb6876cc06ca/attachment.html
../code/2014/581e7b270fa8cb6876cc06ca
successfuly renamed


'../code/2014/581e7b270fa8cb6876cc06ca/attachment.html'

In [42]:
os.stat('../code/2014/581e7b270fa8cb6876cc06c9/attachment')

OSError: [Errno 2] No such file or directory: '../code/2014/581e7b270fa8cb6876cc06c9/attachment'

In [58]:
def build_request_from_folio(folio_document,path_to_file,mongo_request):
    import datetime
    request = {}
    request['folio_id'] = folio_document['_id']
    request['request_url'] = folio_document['ARCHIVORESPUESTA']
    request['date'] = datetime.datetime.now()
    request['path_to_file'] = str(path_to_file)
    result_of_insert = mongo_request.insert_one(request)
    return result_of_insert

In [6]:
np.unique(df['ESTATUS'])

array([u'Desechada por falta de pago',
       u'Desechada por falta de respuesta del ciudadano', u'Terminada'], dtype=object)

In [7]:
df = df.set_index(np.arange(0,len(df)))

In [8]:
print df[['ARCHIVOADJUNTOSOLICITUD']].iloc[3].item()
print df[['ARCHIVORESPUESTA']].iloc[3].item()

https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203
https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeAdjuntoSeguimientoGenerator.action?folio=0000400005203&respuesta=6


In [15]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203','wordDoc','./','.zip')

File is located at: ./wordDoc.zip


In [16]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203','wordDoc','./')

File is located at: ./wordDoc


In [18]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeAdjuntoSeguimientoGenerator.action?folio=0000400005203&respuesta=6','azipfile','./')

File is located at: ./azipfile


In [19]:
respuestas = df['ARCHIVORESPUESTA']
ajuntos = df['ARCHIVOADJUNTOSOLICITUD']
# download only 10 files
respuestas = respuestas[0:10]
# download only 10 files
ajuntos = ajuntos[0:10]

#download_stack_urls(respuestas,'arespuesta','./attachments/respuesta/')

In [20]:
ajuntos_clean = []
for url in ajuntos:
    if str(url) == 'nan':
        continue
    else:
        ajuntos_clean.append(url)
#download_stack_urls(ajuntos_clean,'aajunto','./attachments/ajunto/')

In [21]:
df2 = df[df['attach_inc'] == 1]
max(df2['word_count'])

501358.0

In [22]:
df2 = read_mongo('polisci_test26Oct16','folios',{'attach_inc' : 0},400000)

Connected successfully!!!


In [23]:
len(df2[df2['attach_full']== '    '])

400000

In [28]:
not_attachments = mongo_count('polisci_test26Oct16','folios',{'attach_inc' : 0})
total = mongo_count('polisci_test26Oct16','folios')
print "difference",total-not_attachments
print "percent of documents with attachments included",float(not_attachments)/float(total),'%'

Connected successfully!!!
Connected successfully!!!
difference 128692
percent of documents with attachments included 0.898840086655 %
